In [1]:
import requests
import regex as re
import os
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import warnings

from   sklearn.gaussian_process         import GaussianProcessRegressor
from   sklearn.gaussian_process.kernels import RationalQuadratic
from   scipy.spatial.distance           import cdist
from   sklearn.preprocessing            import MinMaxScaler, RobustScaler

rndm_no = 42
np.random.seed(rndm_no) #set numpy random seed
R_m = 3389.5 #volumetric mean radius
maxRescale = 100
sampSize = 28 #int(np.quantile(jh_all.groupby(by = 'orb')['orb'].count(), 0.25))

lowAlp  = 1e-10
midAlp  = 1e-5
highAlp = 0.5


In [2]:
#https://naif.jpl.nasa.gov/pub/naif/MAVEN/kernels/spk/

baseURL = 'https://naif.jpl.nasa.gov/pub/naif/MAVEN/kernels/spk/'

r = requests.get(baseURL)

x = re.findall('"(maven_orb_rec_.*\.orb)"', r.text)

x.sort()

#os.mkdir('./tempFiles_orb/'), watch out, spaghetti code

In [3]:
orbs      = np.zeros(0)
apoDates = np.zeros(0)


for link in x:
    
    r = requests.get(baseURL+link)
    
    for line in r.text.splitlines()[2:]:
        
        
        orb = line.split()[0]
        
        dateStr = line.split()[6] + line.split()[7]  + line.split()[8]  + '-' + line.split()[9]
        
        apoDate = dt.datetime.strptime(dateStr, "%Y%b%d-%H:%M:%S")
        
        orbs = np.append(orbs, np.int32(orb))
        
        apoDates = np.append(apoDates, apoDate)
        

In [4]:
#get JH file

jh_file = 'https://homepage.physics.uiowa.edu/~jhalekas/drivers/drivers_merge_l2_hires.txt'

r = requests.get(jh_file)

with open("./tempFiles_orb/drivers_merge_l2_hires.txt", "wb") as f:
    
    f.write(r.content)

In [6]:
#read in Halekas file (direct measurements)


colNames = ['date', 'np_SW', 'nalpha_SW', 'v_SW', 
            'v_x_SW', 'v_y_SW', 'v_z_SW', 'tp_SW', 
            'b_x_SW', 'b_y_SW', 'b_z_SW']

jh = pd.read_csv(jh_file, names = colNames, index_col = False, sep = '\s+')

jh['date_SW'] = pd.to_datetime(jh['date'])

jh['date_SW_unix'] = (jh['date_SW'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

jh['b_mag_SW'] = np.sqrt(jh.b_x_SW**2.0 + jh.b_y_SW**2.0 + jh.b_z_SW**2.0)

jh['v_mag_SW'] = np.sqrt(jh.v_x_SW**2.0 + jh.v_y_SW**2.0 + jh.v_z_SW**2.0)


#jh['orb'] = np.nan

#jh_subset   = jh[(jh.date_SW >= startDate) & (jh.date_SW < stopDate)].copy()

#for i, row in jh_subset.iterrows():

#    jh_subset.loc[i, 'date_SW_unix'] = row.date_SW.timestamp()
    
    #which orbit
    
#    jh_subset.loc[i, 'orb'] = orbs[row.date_SW <= apoDates][0]


In [7]:
#log scale the proton temperature and density

jh['ln_np_SW']     = np.log(jh['np_SW'])
jh['ln_tp_SW']     = np.log(jh['tp_SW'])
jh['ln_v_x_SW']    = np.log(-1*jh['v_x_SW'])
jh['ln_v_mag_SW']  = np.log(jh['v_mag_SW'])
jh['ln_b_mag_SW']  = np.log(jh['b_mag_SW'])

In [8]:
params      = ['b_x_SW',     'b_y_SW',    'b_z_SW',   'ln_b_mag_SW',
               'ln_v_x_SW',  'v_y_SW',    'v_z_SW',   'ln_v_mag_SW', 
               'ln_tp_SW',   'ln_np_SW']

shortParams = ['b_x_SW',     'b_y_SW',    'b_z_SW',   'b_mag_SW',
               '-v_x_SW',     'v_y_SW',    'v_z_SW',   'v_mag_SW', 
               'tp_SW',      'np_SW']

In [122]:
#find which orbits we need to run this over

#this denotes where the file runs over the entire time

startDate = dt.datetime(2014, 11, 15)
stopDate = dt.datetime(2014,  11, 16)

orbStartLoop = orbs[apoDates >= startDate][0]

orbEndLoop   = orbs[apoDates >= stopDate][0]

print(orbStartLoop, orbEndLoop)


251.0 257.0


In [123]:
#only currently set up for hourly to save sanity for testing

results = pd.DataFrame()

results['date_[utc]']  = pd.to_datetime(np.arange(startDate,              
                                                  stopDate,             
                                                  dt.timedelta(hours = 1)))

results['date_[unix]']  = (results['date_[utc]'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

results['orb']         = np.nan

for sp in shortParams:

    results['mu_{}'.format(sp)]            = np.nan

    results['sigma_{}'.format(sp)]         = np.nan
    
    results['mu_{}_normed'.format(sp)]     = np.nan

    results['sigma_{}_normed'.format(sp)]  = np.nan
    
    results['flag_{}'.format(sp)]          = np.zeros(len(results))


In [124]:
%%time

#can only go from 1 to -1, this is a little weird for arbitrary dates, will have an edge issue
#will want to fix this for future iterations

#user puts in date, and buffer on either side instead on the search function from jh_all

print('Running from Orbit: {} to {}'.format(orbStartLoop, orbEndLoop))

for i, o in enumerate(np.arange(orbStartLoop, orbEndLoop, 1)):
    
    print(i, o)
    
    endOrb     = apoDates[orbs == o][0]
    startOrb   = apoDates[orbs == o - 1][0]
    midOrb     = startOrb + ((endOrb - startOrb) / 2.0)
    
    #print(startOrb, endOrb)
    
    #get index for table of output
    indResults = ((results['date_[utc]'] >= startOrb) & (results['date_[utc]'] < endOrb))

    results.loc[indResults, 'orb'] = o

    #what do we actually want. EITHER the orbit if it exists +/- 2 - do this first.
    
    indJH = ((jh['date_SW'] >= startOrb) & (jh['date_SW'] < endOrb))
    
    if (sum(indJH) > 0):
        
        #print("Within the dataframe")
    
        startIndex = jh[indJH].index[0]  - 2
        endIndex   = jh[indJH].index[-1] + 2 
        
        
        #print(startIndex, endIndex)

        orbIndex = ((jh.index >= startIndex) & (jh.index <= endIndex))
        

        #if orbital sample too small?
        if sum(orbIndex) < sampSize:
        
            newDelta = int((sampSize - sum(orbIndex)) / 2.0)
            
            #print(newDelta)
        
            startIndex = jh[orbIndex].index[0]  - newDelta
            endIndex   = jh[orbIndex].index[-1] + newDelta
            
            orbIndex = ((jh.index >= startIndex) & (jh.index <= endIndex)) 
            
            #print(sum(orbIndex))
            
    #or the nearest X points to that actual date...

    else:
        
        #amount on either side that scales to the gap distance
        
        #closest index to the gap
        indEnd   = jh.index[((jh.date_SW - endOrb) >= dt.timedelta(0))][0]
        indStart = indEnd - 1
        
        gapStart = jh.loc[indStart, 'date_SW']
        gapEnd   = jh.loc[indEnd, 'date_SW']
        
        fracLoc = ((midOrb - gapStart) / (gapEnd - gapStart))
        
        
        
        newDeltaLeft  = int((1.0 - fracLoc)*sampSize)
        newDeltaRight = int(fracLoc*sampSize)
        
        #print(newDeltaLeft, newDeltaRight)
        startIndex = indStart  - newDeltaLeft
        endIndex   = indEnd    + newDeltaRight
        
        orbIndex = ((jh.index >= startIndex) & (jh.index <= endIndex)) 
        
    
    data = jh[orbIndex].copy()
    

    for p, sp in zip(params, shortParams):

        X = data.date_SW_unix.values.reshape(-1, 1)  
        y = data['{}'.format(p)].values.reshape(-1, 1)

        normScaler = RobustScaler()

        normScaler.fit(y)

        mmScaler = MinMaxScaler(feature_range=(0, maxRescale))
        mmScaler.fit(X)

        X_normed = mmScaler.transform(X)#transform to reasonable X scale to match our prior GP guess
        y_normed = (normScaler.transform(y))[:, 0] #reshape for ML input

        dists        = cdist(X_normed, X_normed)

        dists_noZeros = dists[dists != 0]

        minLength = np.quantile(dists_noZeros, 0.0)

        midLength = np.quantile(dists_noZeros, 0.25) #this is hyperparameter

        maxLength = np.quantile(dists_noZeros, 0.75)

        newScale = np.var(y_normed)#np.max(abs(y_normed))


        with warnings.catch_warnings(record=True) as w:
            



            kernelChoice = newScale**2.0 * RationalQuadratic(length_scale = midLength, 
                                                    length_scale_bounds = (minLength, maxLength), 
                                                    alpha = midAlp, 
                                                    alpha_bounds = (lowAlp, highAlp))


            gpr = GaussianProcessRegressor(kernel = kernelChoice, random_state = rndm_no)


            gpr.fit(X_normed, y_normed)


            if w:
                
                results.loc[indResults, 'flag_{}'.format(sp)]  = np.ones(sum(indResults))

                #for warning in w:

                    #print(warning.message)

                    #print('what')
                
                
                    
        #print(p)

        #print(gpr.kernel_)
            

        X_predict = mmScaler.transform(results.loc[indResults, 'date_[unix]'].values.reshape(-1, 1))

        mean_predict, std_predict = gpr.predict(X_predict, return_std = True)


        '''
        X_model   = np.linspace(0, maxRescale, 2000).reshape(-1, 1)
        mean_model, std_model = gpr.predict(X_model, return_std = True)
        
        plt.fill_between(X_model[:, 0], mean_model  - std_model, 
                                    mean_model  + std_model, 
                                    color = 'grey', alpha = 0.5)

        plt.scatter(X_normed[:, 0], y_normed, color = 'k')

        plt.scatter(X_predict[:, 0], mean_predict, color = 'r')

        plt.plot(X_model[:, 0], mean_model)


        plt.show()

        '''
        results.loc[indResults, 'mu_{}_normed'.format(sp)]     = mean_predict

        results.loc[indResults, 'sigma_{}_normed'.format(sp)]  = std_predict
            

        if 'ln' in p:

            results.loc[indResults, 'mu_{}'.format(sp)]         = np.e**normScaler.inverse_transform(mean_predict.reshape(-1, 1))[:, 0]

            err_x                                               = std_predict*normScaler.scale_

            partial_dqdx                                        = np.e**normScaler.inverse_transform(mean_predict.reshape(-1, 1))[:, 0]

            results.loc[indResults, 'sigma_{}'.format(sp)]      = partial_dqdx*err_x

        else:

            results.loc[indResults, 'mu_{}'.format(sp)] = normScaler.inverse_transform(mean_predict.reshape(-1, 1))[:, 0]

            results.loc[indResults, 'sigma_{}'.format(sp)]  = std_predict*normScaler.scale_
    

Running from Orbit: 251.0 to 257.0
0 251.0
1 252.0
2 253.0
3 254.0
4 255.0
5 256.0
CPU times: user 6.45 s, sys: 41.4 ms, total: 6.49 s
Wall time: 4.62 s
